In [23]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, LSTM, Dropout, Activation, Embedding, Bidirectional
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [24]:
import nltk
from nltk.tokenize import word_tokenize
nltk.download('popular')

[nltk_data] Downloading collection 'popular'
[nltk_data]    | 
[nltk_data]    | Downloading package cmudict to
[nltk_data]    |     C:\Users\BJ\AppData\Roaming\nltk_data...
[nltk_data]    |   Package cmudict is already up-to-date!
[nltk_data]    | Downloading package gazetteers to
[nltk_data]    |     C:\Users\BJ\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gazetteers is already up-to-date!
[nltk_data]    | Downloading package genesis to
[nltk_data]    |     C:\Users\BJ\AppData\Roaming\nltk_data...
[nltk_data]    |   Package genesis is already up-to-date!
[nltk_data]    | Downloading package gutenberg to
[nltk_data]    |     C:\Users\BJ\AppData\Roaming\nltk_data...
[nltk_data]    |   Package gutenberg is already up-to-date!
[nltk_data]    | Downloading package inaugural to
[nltk_data]    |     C:\Users\BJ\AppData\Roaming\nltk_data...
[nltk_data]    |   Package inaugural is already up-to-date!
[nltk_data]    | Downloading package movie_reviews to
[nltk_data]    |     C:\Users

True

In [25]:
dataset = []
nursery_rhyme = []

with open("./datasets/nursery_rhymes.txt", 'r') as f:
    lines = f.readlines()
    for line in lines:
        if not line.startswith(" ") and line != "\n" and nursery_rhyme: # if the line is a title, reset to new nursery_rhyme
            dataset.append(" ".join(nursery_rhyme))
            nursery_rhyme = []
        parsed = line.strip(" \n!;.,_")
        if parsed: # remove solo new line characters
            nursery_rhyme.extend(parsed.lower().split())
print(dataset)

["the queen _of_ hearts the queen of hearts she made some tarts all on a summer's day the knave of hearts he stole those tarts and took them clean away the king of hearts called for those tarts and beat the knave full sore the knave of hearts brought back those tarts and vowed he'd steal no more", "saint swithin's day st. swithin's day, if thou dost rain for forty days it will remain st. swithin's day, if thou be fair for forty days 't will rain no more", 'dance to your daddie dance to your daddie my bonnie laddie dance to your daddie, my bonnie lamb you shall get a fishie on a little dishie you shall get a fishie when the boat comes hame dance to your daddie my bonnie laddie dance to your daddie, and to your mammie sing you shall get a coatie and a pair of breekies you shall get a coatie when the boat comes in', 'the man in the moon the man in the moon came tumbling down and asked the way to norwich he went by the south and burnt his mouth with eating cold pease porridge', 'simple sim

In [26]:
train_size = .8
total_size = len(dataset)
train_data = dataset[:int(train_size*total_size)]
test_data = dataset[int(train_size*total_size):]
print("total size: ", total_size)
print("train size: ", len(train_data))
print("test size: ", len(test_data))

total size:  313
train size:  250
test size:  63


In [27]:
oov_tok = "<OOV>"

In [28]:
tokenizer = Tokenizer(num_words = 5000, oov_token=oov_tok)
tokenizer.fit_on_texts(train_data)
word_index = tokenizer.word_index
train_data = tokenizer.texts_to_sequences(train_data)
train_data
    

[[2,
  145,
  10,
  355,
  2,
  145,
  10,
  355,
  16,
  64,
  78,
  424,
  18,
  23,
  4,
  727,
  79,
  2,
  539,
  10,
  355,
  6,
  270,
  425,
  424,
  3,
  120,
  33,
  540,
  66,
  2,
  109,
  10,
  355,
  214,
  21,
  425,
  424,
  3,
  146,
  2,
  539,
  305,
  728,
  2,
  539,
  10,
  355,
  541,
  195,
  425,
  424,
  3,
  542,
  729,
  543,
  147,
  196],
 [1121,
  544,
  79,
  236,
  544,
  79,
  59,
  215,
  730,
  545,
  21,
  731,
  271,
  17,
  27,
  1122,
  236,
  544,
  79,
  59,
  215,
  45,
  237,
  21,
  731,
  271,
  356,
  27,
  545,
  147,
  196],
 [181,
  5,
  46,
  357,
  181,
  5,
  46,
  357,
  19,
  546,
  732,
  181,
  5,
  46,
  357,
  19,
  546,
  547,
  11,
  52,
  68,
  4,
  733,
  23,
  4,
  7,
  1123,
  11,
  52,
  68,
  4,
  733,
  30,
  2,
  734,
  238,
  1124,
  181,
  5,
  46,
  357,
  19,
  546,
  732,
  181,
  5,
  46,
  357,
  3,
  5,
  46,
  1125,
  69,
  11,
  52,
  68,
  4,
  735,
  3,
  4,
  272,
  10,
  1126,
  11,
  52,
  68,
  4,
  73

In [ ]:
train_padded = pad_sequences(train_data, maxlen=600, padding='post', truncating='post')

In [57]:
def generate_padded_sequences(input_sequences):
    max_sequence_len = max([len(x) for x in input_sequences])
    input_sequences = np.array(pad_sequences(input_sequences, maxlen=max_sequence_len, padding='post'))
    predictors, label = input_sequences[:,:-1],input_sequences[:,-1]
    label = ku.to_categorical(label, num_classes=total_words)
    return predictors, label, max_sequence_len

# predictors, label, max_sequence_len = generate_padded_sequences(inp_sequences)

In [58]:
model = Sequential()
model.add(Embedding(vocab_size, embedding_dim))
model.add(Dropout(0.5))
model.add(Bidirectional(LSTM(embedding_dim)))
model.add(Dense(6, activation='softmax'))
model.summary()

NameError: name 'vocab_size' is not defined